# 2th_homework

### Import Section

In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix

from implicit.nearest_neighbours import CosineRecommender
from implicit.nearest_neighbours import TFIDFRecommender

### Global Settings Section

In [2]:
# random_state_global = 0

### Path Section

In [3]:
PATH_DATA_RETAIL = r'retail_train.csv'

## 0. Data Preparation

#### Loading DataFrame

In [4]:
df_retail = pd.read_csv(PATH_DATA_RETAIL)

In [5]:
df_retail.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


#### Train Test Split

In [6]:
test_size_weeks = 3
columns_to_split = ['user_id', 'item_id', 'sales_value', 'quantity']

df_train = df_retail.loc[df_retail['week_no'] < df_retail['week_no'].max() - test_size_weeks, columns_to_split]
df_test = df_retail.loc[df_retail['week_no'] >= df_retail['week_no'].max() - test_size_weeks, columns_to_split]

In [7]:
print(f'Размерность тренировочного набора данных:\t{df_train.shape}' \
      f'\nРазмерность валидационного набора данных:\t{df_test.shape}')

Размерность тренировочного набора данных:	(2278490, 4)
Размерность валидационного набора данных:	(118314, 4)


In [8]:
df_train.head()

,user_id,item_id,sales_value,quantity
0,2375,1004906,1.39,1
1,2375,1033142,0.82,1
2,2375,1036325,0.99,1
3,2375,1082185,1.21,1
4,2375,8160430,1.50,1


### df_results

Создание набора данных для валидации.

In [9]:
df_results = (
    df_test
    .groupby(by='user_id')['item_id']
    .unique()
    .reset_index()
    .rename(columns={'item_id': 'actual'})
)

In [10]:
df_results.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


### array_items

Создание массива всех товаров.

In [11]:
array_items = df_train['item_id'].unique()

In [12]:
array_items

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393],
      dtype=int64)

In [13]:
len(array_items)

86865

## 1. weighted_random_recommendation

Реализация функции случайного семплирования с применением весов.

In [14]:
# Функция для семплирования случайных товаров, но с учётом весов.
def weighted_random_recommendation(items, items_weight=None, n=5):
    # Если веса не заданы, но будут использоваться равные веса.
    if items_weight is None:
        return np.random.choice(items, size=n, replace=False)
    
    return np.random.choice(items, size=n, p=items_weight, replace=False)

Проверка работы функции без указания весов товаров.

In [15]:
df_results_rr = df_results.copy()

In [16]:
%%time

df_results_rr['random_recommendation'] = (
    df_results_rr['user_id']
    .apply(lambda x: weighted_random_recommendation(items=array_items))
)

Wall time: 2.7 s


In [17]:
df_results_rr.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1121911, 2980579, 929815, 6602341, 8381265]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[833591, 942650, 657576, 1731605, 12185466]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1354189, 2541140, 1828522, 990444, 6602487]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[13876458, 1691935, 15973062, 1082614, 994791]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1011215, 968478, 1008324, 1852773, 355985]"


Для каждого пользователя были выбраны пять случайных товаров.

Сгенерируем веса для выбора товаров на основе их популярности: количество уникальных пользователей для каждого товара.

In [18]:
# Формирование набора данных из двух строк:
# item_id - уникальный идентификатор товара;
# total_users - количество уникальных пользователей, которые приобрели данный товар.
array_items_weights = (
    df_train
    .groupby(by='item_id')['user_id']
    .nunique()
    .reset_index()
    .rename(columns={'user_id': 'total_users'})
)

# Перевод столбца "total_users" в вектор Numpy.
array_items_weights = np.array(array_items_weights['total_users'])
# Вычисление суммы всех элементов вектора для нормализации.
items_weights_total = array_items_weights.sum()
# Нормализация вектора.
array_items_weights = array_items_weights / items_weights_total

In [19]:
array_items_weights

array([2.38091837e-06, 7.93639456e-07, 7.93639456e-07, ...,
       7.93639456e-07, 7.93639456e-07, 1.58727891e-06])

Вызов функции с указанием весов.

In [20]:
df_results_wrr = df_results.copy()

In [21]:
%%time

df_results_wrr['weighted_random_recommendation'] = (
    df_results_wrr['user_id']
    .apply(lambda x: weighted_random_recommendation(items=array_items, items_weight=array_items_weights))
)

Wall time: 1.27 s


In [22]:
df_results_wrr.head()

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5565680, 894123, 97991, 9527124, 6396315]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5805582, 834681, 5590390, 937460, 12330150]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6979161, 863578, 10119913, 469747, 1521353]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1059292, 1019367, 873969, 1059665, 503913]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[894358, 9935348, 1134840, 15596991, 1759399]"


Теперь товары были выбраны с учётом их популярности у пользователей: товары с большим числом уникальных пользователей имели более высокие шансы попасть в выборку.

## 2. Можно ли улучшить бейзлайны, если считать их на топ-5000 товарах? - это значит, что мы должны не просто забивать товары не из ТОП-5000 одним индексом 999999 и учиться на всех данных. А учиться только на взаимодействиях с товарами из ТОП5000

#### Top 5000 items

Выбор первых 5000 наиболее популярных товаров: товаров с наибольшими объёмами продаж.

In [23]:
array_items_top_5000 = (
    df_train
    .groupby(by='item_id')['sales_value']
    .sum()
    .reset_index()
    .sort_values(by='sales_value', ascending=False)
    .head(5000)['item_id']
    .unique()
)

In [24]:
array_items_top_5000

array([6534178, 6533889, 1029743, ...,  981716, 5575861,  915174],
      dtype=int64)

In [25]:
len(array_items_top_5000)

5000

#### Metric "Precision@k"

Метрика качества: точность прогноза для первых k товаров.

In [26]:
def precision_at_k(actual, predicted, k=5):
    
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    return np.isin(predicted[:k], actual).sum() / k

### weighted_random_recommendation

Выполним прогноз для топ 5000 товаров с весами и без.

Сформируем веса для топ 5000 товаров.

In [27]:
# Формирование набора данных из двух строк:
# item_id - уникальный идентификатор товара;
# total_users - количество уникальных пользователей, которые приобрели данный товар.
array_items_weights_top_5000 = (
    df_train
    .groupby(by='item_id')['user_id']
    .nunique()
    .reset_index()
    .rename(columns={'user_id': 'total_users'})
)

# Отбор товаров, которые присутствуют в списке наиболее популярных.
array_items_weights_top_5000 = array_items_weights_top_5000[
    array_items_weights_top_5000['item_id'].isin(array_items_top_5000)
]
# Перевод столбца "total_users" в вектор Numpy.
array_items_weights_top_5000 = np.array(array_items_weights_top_5000['total_users'])
# Вычисление суммы всех элементов вектора для нормализации.
items_weights_total_top_5000 = array_items_weights_top_5000.sum()
# Нормализация вектора.
array_items_weights_top_5000 = array_items_weights_top_5000 / items_weights_total_top_5000

Выполним семплирование для топ 5000 товаров без учёта весов.

In [28]:
%%time

df_results_rr['random_recommendation_top_5000'] = (
    df_results_rr['user_id']
    .apply(lambda x: weighted_random_recommendation(array_items_top_5000))
)

Wall time: 182 ms


In [29]:
df_results_rr.head()

,user_id,actual,random_recommendation,random_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1121911, 2980579, 929815, 6602341, 8381265]","[1084551, 6534074, 12525270, 1077975, 851865]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[833591, 942650, 657576, 1731605, 12185466]","[894140, 1113111, 852572, 1048727, 847270]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1354189, 2541140, 1828522, 990444, 6602487]","[5564067, 950463, 995965, 999270, 1065556]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[13876458, 1691935, 15973062, 1082614, 994791]","[846811, 972418, 944256, 9487885, 846986]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1011215, 968478, 1008324, 1852773, 355985]","[899624, 6633031, 5568489, 1052752, 7410071]"


Выполним семплирование для топ 5000 товаров с учётом весов.

In [30]:
%%time

df_results_wrr['weighted_random_recommendation_top_5000'] = (
    df_results_wrr['user_id']
    .apply(lambda x: weighted_random_recommendation(array_items_top_5000, array_items_weights_top_5000))
)

Wall time: 171 ms


In [31]:
df_results_wrr.head()

,user_id,actual,weighted_random_recommendation,weighted_random_recommendation_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5565680, 894123, 97991, 9527124, 6396315]","[7440663, 1037965, 863030, 942565, 945652]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[5805582, 834681, 5590390, 937460, 12330150]","[1109206, 1124695, 6979253, 1095393, 908514]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6979161, 863578, 10119913, 469747, 1521353]","[5585701, 1124694, 1041407, 880469, 9653535]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1059292, 1019367, 873969, 1059665, 503913]","[1025435, 870211, 1059473, 9416653, 862488]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[894358, 9935348, 1134840, 15596991, 1759399]","[12263788, 938512, 7441391, 910439, 868548]"


Посчитаем метрики для всех товаров и для топ 5000.

In [32]:
%%time

df_metrics = pd.DataFrame(
    data=[[(df_results_rr['random_recommendation']
            .apply(lambda row: precision_at_k(df_results_rr['actual'], row), 1)
            .mean()),
          (df_results_rr['random_recommendation_top_5000']
           .apply(lambda row: precision_at_k(df_results_rr['actual'], row), 1)
           .mean())]],
    index=['random_recommendation'],
    columns=['all_items', 'top_5000']
)

Wall time: 1min 21s


In [33]:
%%time

df_metrics.loc['weighted_random_recommendation', :] = [
    (df_results_wrr['weighted_random_recommendation']
         .apply(lambda row: precision_at_k(df_results_wrr['actual'], row), 1)
         .mean()),
    (df_results_wrr['weighted_random_recommendation_top_5000']
         .apply(lambda row: precision_at_k(df_results_wrr['actual'], row), 1)
         .mean())
]

Wall time: 1min 22s


In [34]:
df_metrics

,all_items,top_5000
random_recommendation,0.000294,0.002547
weighted_random_recommendation,0.000490,0.003624


Оба метода демонстрируют посредственные метрики точности. Однако можно наблюдать тенденцию, что применение весов положительно сказывается на точности прогноза, как и применение выборки наиболее покупаемых товаров.

### CosineRecommender

Для оценки эффективности базовых алгоритмов выполним прогнозирование при помощи рекомендательных моделей.

Выполним прогнозирование на топ 5000 товарах, остальные товары будут проигнорированы фильтром модели.

Далее идёт код из методички с незначительными изменениями.

In [35]:
df_results_cr = df_results[['user_id', 'actual']].copy()

In [36]:
%%time

df_train_cr = df_train.copy()
df_train_cr.loc[~df_train_cr['item_id'].isin(array_items_top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(df_train_cr, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1
user_item_matrix = user_item_matrix.astype(float)

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

Wall time: 2.06 s


In [37]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [38]:
%%time

model_cr = CosineRecommender(K=5, num_threads=0)

model_cr.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 294 ms


In [39]:
%%time

df_results_cr['cosine_recommender'] = df_results_cr['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_cr.recommend(userid=userid_to_id[x],
                                       user_items=sparse_user_item,
                                       N=5,
                                       filter_already_liked_items=False,
                                       filter_items=[itemid_to_id[999999]],
                                       recalculate_user=True)])

Wall time: 48.9 ms


In [40]:
df_results_cr.head()

,user_id,actual,cosine_recommender
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 1098066, 981760, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 981760, 1127831, 1098066, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 981760, 1127831, 961554, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 981760, 1098066, 1127831, 826249]"


In [41]:
%%time

df_metrics.loc['cosine_recommender', :] = [
    np.NaN,
    (df_results_cr['cosine_recommender'].apply(lambda row: precision_at_k(df_results_cr['actual'], row), 1).mean())
]

Wall time: 41.4 s


In [42]:
df_metrics

,all_items,top_5000
random_recommendation,0.000294,0.002547
weighted_random_recommendation,0.000490,0.003624
cosine_recommender,NaN,0.047894


Прогнозирование на всём объёме товаров затруднительно из-за ресурсоёмкости операции.

Модель, использующая косинусную меру для подбора похожих товаров, демонстрирует гораздо лучшую метрику качества.

### TFIDFRecommender

Выполним прогноз для топ 5000 товаров при помощи модели инверсии частоты.

In [43]:
df_results_tfidf = df_results[['user_id', 'actual']].copy()

In [44]:
%%time

model_tfidf = TFIDFRecommender(K=5, num_threads=0)

model_tfidf.fit(csr_matrix(user_item_matrix).T.tocsr(),
                show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 269 ms


In [45]:
%%time

df_results_tfidf['tfidf'] = df_results_tfidf['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in
                     model_tfidf.recommend(userid=userid_to_id[x],
                                           user_items=sparse_user_item,
                                           N=5,
                                           filter_already_liked_items=False,
                                           filter_items=[itemid_to_id[999999]],
                                           recalculate_user=False)])

Wall time: 57.8 ms


In [46]:
df_results_tfidf.head()

,user_id,actual,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 981760, 1127831, 1098066, 961554]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 981760, 1098066, 826249, 883404]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1082185, 981760, 1127831, 878996, 961554]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1082185, 981760, 1127831, 961554, 826249]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1082185, 981760, 1098066, 826249, 1127831]"


In [47]:
%%time

df_metrics.loc['tfidf', :] = [
    np.NaN,
    (df_results_tfidf['tfidf'].apply(lambda row: precision_at_k(df_results_tfidf['actual'], row), 1).mean())
]

Wall time: 41.8 s


In [48]:
df_metrics

,all_items,top_5000
random_recommendation,0.000294,0.002547
weighted_random_recommendation,0.000490,0.003624
cosine_recommender,NaN,0.047894
tfidf,NaN,0.046131


Модель, использующая инверсию частоты для прогнозирования, демонстрирует схожие результаты с моделью, использующей косинусную меру. При этом обе модели демонстрируют значительно лучшие результаты, чем случайное семплирование с весами и без.

Выполнить прогнозирование на всех объектах также затруднительно из-за русорсоёмкости операции.